## Word2Vec testings

##### Have defined 3 functions for the performing the word2vec and feature selection using the same efficiently, with the flexibility of selecting the threshold for the amount of words to be considered as a deciding factor for the feature selection.

In [1]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.word2vec import H2OWord2vecEstimator
import random, os, sys
from datetime import datetime
import pandas as pd
import csv
import time
from distutils.util import strtobool
import psutil
import numpy as np
from matplotlib import pyplot as plt
import statistics
import warnings
warnings.filterwarnings("ignore")

In [2]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

5


In [3]:
h2o.init(strict_version_check=False,max_mem_size=min_mem_size)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\prabh\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\prabh\AppData\Local\Temp\tmpu61apygv
  JVM stdout: C:\Users\prabh\AppData\Local\Temp\tmpu61apygv\h2o_prabh_started_from_python.out
  JVM stderr: C:\Users\prabh\AppData\Local\Temp\tmpu61apygv\h2o_prabh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.12.0.1
H2O cluster version age:,"1 year, 11 months and 3 days !!!"
H2O cluster name:,H2O_from_python_prabh_yhgumz
H2O cluster total nodes:,1
H2O cluster free memory:,4.445 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [4]:
data_path = "data/AmazonReviews.Small.csv"
reviews = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
print(reviews.shape)
print(reviews.head(3))

(55554, 10)


ProductId,UserId,Summary,Score,HelpfulnessDenominator,Id,ProfileName,HelpfulnessNumerator,Time,Text
B00141QYSQ,A1YS02UZZGRDCT,Do Not Buy,1,2,41471,Evan Eberhardt,2,1.34836e+09,"These are made in China (do not buy ANY pet food from China). Dogswell has been using propylene glycol to soften their treats (what are they thinkng?). Do not purchase or support this company in any way until they clean up their act. And for whatever reason Amazon doesn't allow returns of this item, so I had to toss mine out. Bad business all around on this one."
B0089SPEO2,A3JOYNYL458QHP,Less lemon and less zing,3,0,28582,coleridge,0,1.32391e+09,"Everything is ok, except it just isn't as good as it is in the bags. Just considerably more bland -- less lemon and less zing. Boring."
B001PMCDK2,A14TTMM0Z03Y2W,my cat goes crazy for these!,5,0,389965,Lindsay S. Bradford,0,1.3106e+09,"Best cat treat ever. There isn't anything comparable to the love my cat has for these treats, he snubs away any other kind now.I know he likes to manipulate me with his cattiness but these treats are my way of manipulating him to come sit on my lap and have some chill time. :)"


##### Below are the STOP_WORDS that will be useful in the analysis. The location for the same to be downloaded is as given
https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv

In [6]:
# Stopwords for the testings is included
# Below code tests if Stopping word is present in the local machine, if not then it finds it at the specific GitHub 

data_path = "data/stopwords.csv"
if os.path.isfile(data_path):
    data_path = data_path
else:
    data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

### Functions

Few functions that has been build to work with the testings. Below are the list and explanation for the same of it's working. With the input and return explained for the functions

* <b>w2v_col</b> - This function helps us get the list of all the columns which has the text above a threshold that is given as a input `w_threshold`. And the data frame should be `pandas df` data frame given as an input for it to find it.
* <b>w2v_col_avg</b> - This function helps us get the list of all the columns which has the text above a threshold average count of characters in the column that is given as an input `w_threshold`. And the data frame should be `pandas df` data frame given as an input for it to find it.
* <b>tokenize</b> - This function goes through the text string in each columns and tokenize it to words based on the STOP_WORDS that is given to it. This is used and called in the 3rd function below
* <b>word2vec_func_df</b> - This function helps us get the data frame with all the word converted to vector and aggregated with the `AVERAGE` method. This takes output of first function which is the list of all columns to be tokenized and takes the threshold amount of columns to be vecotrized in the result for each of the columns being considered, also takes the H2O Dataframe as an input for tokenizing and vectoring it. As the output it returns the combined data frame consisting all the words into vectors for our further analysis

In [7]:
# A function to return the columns that has some threshold amount of words, 
# so that they can be considered for word2vec training and adding as features to the dataset
def w2v_col(df, w_threshold):
    do_w2v=0
    w2v_col=[]
    for col in df.columns:
        do_w2v=0
        try:
            for val in df[col].str.len():
                if val > w_threshold:
                    do_w2v=1
            if do_w2v==1:
                w2v_col.append(col)
        except:
            print("Skipped: "+col)
    print("List of text field above "+str(w_threshold)+": "+str(w2v_col))
    return w2v_col

# Second version of the function considering the average count of characters in the column
def w2v_col_avg(df, w_threshold):
    do_w2v=0
    w2v_col=[]
    for col in df.columns:
        do_w2v=0
        try:
            if int(statistics.mean(df[col].str.len()))>=w_threshold:
                do_w2v=1
                w2v_col.append(col)
                print("Min count of characters = "+ str(min(df['Text'].str.len())) + " in {} column of the dataset".format('Text'))
                print("Max count of characters = "+ str(max(df['Text'].str.len())) + " in {} column of the dataset".format('Text'))
                print("Avg count of characters = "+ str(int(statistics.mean((df['Text'].str.len())))) + " in {} column of the dataset".format('Text'))
                print("***************************")
        except:
            print("Skipped: "+col)
    print("List of text field above "+str(w_threshold)+": "+str(w2v_col))
    return w2v_col

# Function to tockenize the words compared to the stop words
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

# word2vec training and getting a dataset as a final with all the vectors binded with the original dataset
def word2vec_func_df(arr, vec_size, df):
    for col in arr:
        words = tokenize(df[col].ascharacter())
        model_id= "w2v_"+col+".hex"
        w2v_model = H2OWord2vecEstimator(vec_size = vec_size, model_id = model_id)
        w2v_model.train(training_frame=words)
        models_path='/data'
        h2o.save_model(w2v_model, path = models_path, force = True)
        print(w2v_model.find_synonyms("coffee", count = 10))
        print(w2v_model.find_synonyms("tea", count = 10))
        df_vec = w2v_model.transform(words, aggregate_method = "AVERAGE")
        df_vec.names = [col + s for s in df_vec.names]
        ext_df = df.cbind(df_vec)
        df = ext_df
    return ext_df

In [8]:
# Used pandas to find the columns that have a threshold amount of words to be considered

df=pd.read_csv("data/AmazonReviews.Small.csv")
w2vcol=w2v_col_avg(df, 100)
w2vcol

Skipped: Summary
Skipped: Score
Skipped: HelpfulnessDenominator
Skipped: Id
Skipped: HelpfulnessNumerator
Skipped: Time
Min count of characters = 32 in Text column of the dataset
Max count of characters = 11861 in Text column of the dataset
Avg count of characters = 434 in Text column of the dataset
***************************
List of text field above 100: ['Text']


['Text']

In [9]:
reviews["PositiveReview"] = (reviews["Score"] >= 4).ifelse("1", "0")

In [10]:
reviews.head()

ProductId,UserId,Summary,Score,HelpfulnessDenominator,Id,ProfileName,HelpfulnessNumerator,Time,Text,PositiveReview
B00141QYSQ,A1YS02UZZGRDCT,Do Not Buy,1,2,41471,Evan Eberhardt,2,1.34836e+09,"These are made in China (do not buy ANY pet food from China). Dogswell has been using propylene glycol to soften their treats (what are they thinkng?). Do not purchase or support this company in any way until they clean up their act. And for whatever reason Amazon doesn't allow returns of this item, so I had to toss mine out. Bad business all around on this one.",0
B0089SPEO2,A3JOYNYL458QHP,Less lemon and less zing,3,0,28582,coleridge,0,1.32391e+09,"Everything is ok, except it just isn't as good as it is in the bags. Just considerably more bland -- less lemon and less zing. Boring.",0
B001PMCDK2,A14TTMM0Z03Y2W,my cat goes crazy for these!,5,0,389965,Lindsay S. Bradford,0,1.3106e+09,"Best cat treat ever. There isn't anything comparable to the love my cat has for these treats, he snubs away any other kind now.I know he likes to manipulate me with his cattiness but these treats are my way of manipulating him to come sit on my lap and have some chill time. :)",1
B002Q8JOSI,A17UQD2RSSQH5X,My dogs tell me these treats are YUMMY,5,1,212536,in the dark,1,1.31613e+09,"My two Corgis were thoroughly spoiled by my late husband (I spent a year and a half dieting them down a combined total of 25 pounds!)They are accustomed to the finest of fare, and they absolutely love the Wellness brand of treats.",1
B00176G870,A2F2MZW8EOGH5J,Yummy to the tummy,5,0,115971,"daemoncycler """"When you arrive at a fork in th..",0,1.33479e+09,We used to have drive down to the specialty pet food store for this product. So glad we discovered Amazon. As far as I can tell it is no more expensive & in some cases less - Prime membership is awesome. Loving Pets treats are some of the best according to my dog. They do not develop that nasty smell like some dog treats do.,1
B001CHFUGY,A2M8VROSDPU4JT,Very good coffee,5,1,434484,"Officefan """"Officefankt",1,1.27725e+09,"I really liked this coffee, it was just as good as everyone claimed it was. Strong, bold and flavorful! I would recommend!",1
B0041CIR62,A16I6WJUEBJ1C3,okay but not as healthy as it appears,2,1,138997,"doctorsirena """"doctorsirena",1,1.34369e+09,"I am always looking for healthier, whole grain versions of foods I enjoy. Unfortunately, these Peacock brand noodles are yet another food masquerading as healthy. The product title in big letters on the front says """"Brown Rice Vermicelli"""", making the consumer think """"this is made with brown rice, so it should be a healthy choice"""". But the first indication that it is not is when looking at the fiber content on the nutrition facts - only 0.6g per 2oz serving. Then onto the ingredients list to see why so low... contains brown rice, sago starch and water. The sago starch comes from palms and must not have much (if any) fiber.The Annie Chun's Maifun Brown Rice Noodles (sold on Amazon and in my local healthy grocer) has become one of my staples and is my frame of reference when comparing to the Peacock brand. The Annie Chun's product is made with 100% whole grain, with ingredients brown rice flour and water. Per 2oz serving, it has 4g fiber and pretty much the same calories and other nutrients as the Peacock brand.If you do try this Peacock brand noodles and have not used rice noodles before, you will need to seek guidance elsewhere on preparation. As others have pointed out, the Peacock package gives almost no directions on how to prepare the product, aside from a brief mention in the recipes (in the header text it does say that they are """"easy-to-cook"""" but does not say how). It also contains a very strange recipe for rice noodles: Aglio Olio style - this is an Italian recipe for noodles with olive oil/garlic/sprinkled with grated cheese that I think would not be very tasty. The second recipe appears to be for a soup with veggie strips. Neither recipe gives amoun

We first build a model based on the original dataset to evaluate it's metric over AUC

In [11]:
train,test,valid = reviews.split_frame(ratios=[.7, .15])

In [12]:
from h2o.estimators import H2OGradientBoostingEstimator

predictors = ['ProductId', 'UserId', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time']
response = 'PositiveReview'

gbm_baseline = H2OGradientBoostingEstimator(stopping_metric = "AUC", stopping_tolerance = 0.001,
                                            stopping_rounds = 5, score_tree_interval = 10,
                                            model_id = "gbm_baseline.hex"
                                           )
gbm_baseline.train(x = predictors, y = response, 
                   training_frame = train, validation_frame = test
                  )

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [13]:
print("AUC on Validation Data: " + str(round(gbm_baseline.auc(valid = True), 3)))

AUC on Validation Data: 0.735


##### Used the function defined above to get the new features out of the text fields and appending it to the original dataset for further usage

In [14]:
# Used the function defined above to get the new features out of the texy fields and appending it to the riginal dataset 
# for further usage

ext_reviews=word2vec_func_df(w2vcol, 50, reviews)
ext_reviews

word2vec Model Build progress: |██████████████████████████████████████████| 100%
OrderedDict([('coffe', 0.8779850602149963), ('expresso', 0.8613069653511047), ('espresso', 0.8602164387702942), ('brew', 0.8555266857147217), ('bunn', 0.8278616666793823), ('coffees', 0.8246217370033264), ('brewed', 0.8244238495826721), ('java', 0.8199266791343689), ('cup', 0.8175923824310303), ('starbucks', 0.8047832250595093)])
OrderedDict([('teas', 0.8847182989120483), ('lipton', 0.8423459529876709), ('herbal', 0.8370890617370605), ('oolong', 0.8351590037345886), ('stash', 0.8339065909385681), ('rooibos', 0.8087781667709351), ('chai', 0.8087589740753174), ('herbals', 0.807898998260498), ('genmaicha', 0.7935976982116699), ('chamomile', 0.7916214466094971)])


ProductId,UserId,Summary,Score,HelpfulnessDenominator,Id,ProfileName,HelpfulnessNumerator,Time,Text,PositiveReview,TextC1,TextC2,TextC3,TextC4,TextC5,TextC6,TextC7,TextC8,TextC9,TextC10,TextC11,TextC12,TextC13,TextC14,TextC15,TextC16,TextC17,TextC18,TextC19,TextC20,TextC21,TextC22,TextC23,TextC24,TextC25,TextC26,TextC27,TextC28,TextC29,TextC30,TextC31,TextC32,TextC33,TextC34,TextC35,TextC36,TextC37,TextC38,TextC39,TextC40,TextC41,TextC42,TextC43,TextC44,TextC45,TextC46,TextC47,TextC48,TextC49,TextC50
B00141QYSQ,A1YS02UZZGRDCT,Do Not Buy,1,2,41471,Evan Eberhardt,2,1.34836e+09,"These are made in China (do not buy ANY pet food from China). Dogswell has been using propylene glycol to soften their treats (what are they thinkng?). Do not purchase or support this company in any way until they clean up their act. And for whatever reason Amazon doesn't allow returns of this item, so I had to toss mine out. Bad business all around on this one.",0,0.211648,0.0836178,0.154432,-0.0801733,0.0270559,-0.0451945,0.0126701,0.0397198,0.223853,0.144401,0.281643,0.0160786,-0.128643,-0.239099,0.268369,0.0584515,0.149322,0.151307,0.0653841,-0.241044,0.0475262,-0.369009,0.0813263,0.0227773,-0.180316,0.242687,0.138766,0.163693,0.070553,-0.072239,0.0823268,-0.0737676,0.136185,0.0373692,0.0924752,0.182207,0.0394658,-0.148737,-0.136277,-0.0115398,0.126479,0.0512608,0.371145,0.0446014,-0.133573,0.0821626,-0.0353309,0.00448653,-0.174391,-0.162296
B0089SPEO2,A3JOYNYL458QHP,Less lemon and less zing,3,0,28582,coleridge,0,1.32391e+09,"Everything is ok, except it just isn't as good as it is in the bags. Just considerably more bland -- less lemon and less zing. Boring.",0,-0.123725,0.0515569,-0.0895349,0.0209407,-0.0978972,0.0212541,0.158259,0.0319733,0.129587,0.0303066,0.152216,0.16456,0.0840599,-0.0534468,0.0466499,-0.0885551,0.141814,-0.0223999,0.0711059,-0.318269,0.2734,-0.212646,0.13178,0.239815,0.0276721,0.240725,0.0981239,0.296461,-0.0375646,0.208913,0.18172,-0.191207,0.273754,-0.0374647,0.0974908,0.209961,-0.186399,-0.108684,0.124317,-0.252231,0.133675,-0.0921199,0.112973,0.231362,-0.0250295,0.0655465,0.0221939,-0.036063,-0.0136473,-0.0177618
B001PMCDK2,A14TTMM0Z03Y2W,my cat goes crazy for these!,5,0,389965,Lindsay S. Bradford,0,1.3106e+09,"Best cat treat ever. There isn't anything comparable to the love my cat has for these treats, he snubs away any other kind now.I know he likes to manipulate me with his cattiness but these treats are my way of manipulating him to come sit on my lap and have some chill time. :)",1,0.0889585,-0.04399,0.0617054,-0.0552317,0.0542779,-0.120872,0.0449626,0.194192,0.083394,0.140715,0.41009,0.338975,0.0977968,-0.13574,0.0827583,0.0156031,-0.12947,0.0399429,-0.0547307,-0.19249,-0.0027032,-0.218035,0.0270154,0.010713,-0.129571,0.225542,0.184253,0.180263,0.0322203,-0.0064634,0.148347,-0.143664,0.235323,-0.0556643,0.0302875,0.200281,-0.0124474,-0.201645,0.00668386,-0.0768465,0.181294,-0.105857,0.322995,-0.0284267,0.0183989,0.150104,-0.0355406,-0.0424858,-0.0255324,-0.114745
B002Q8JOSI,A17UQD2RSSQH5X,My dogs tell me these treats are YUMMY,5,1,212536,in the dark,1,1.31613e+09,"My two Corgis were thoroughly spoiled by my late husband (I spent a year and a half dieting them down a combined total of 25 pounds!)They are accustomed to the finest of fare, and they absolutely love the Wellness brand of treats.",1,0.0759095,0.0958257,-0.0127404,-0.148443,0.00606237,-0.266269,0.111488,0.111778,0.0723661,-0.0372775,0.266655,0.287477,0.0995035,-0.0584517,0.0405493,0.0404164,-0.117754,0.174061,0.0115582,-0.268218,0.0677709,-0.242939,0.0190169,0.00168814,-0.0706935,0.115715,-0.0244717,0.206319,0.0117178,0.0885945,0.229826,-0.126863,0.166978,0.0218396,0.0886459,0.107742,-0.0856443,-0.186326,0.0678053,-0.15813,0.108314,0.0105831,0.306114,-0.029519,-0.0861071,0.0676794,0.027092,-0.129595,-0.139358,-0.156354
B00176G870,A2F2MZW8EOGH5J,Yummy to the tummy,5,0,115971,"daemoncycler """"When you arrive at a fork in th..",0,1.33479e+09,We used to have dri

In [15]:
ext_train,ext_test,ext_valid = ext_reviews.split_frame(ratios=[.7, .15])

### We build the model over the dataset consisting of all the word2vec as a features. And test how it improves our AUC

In [16]:
predictors = predictors + ext_reviews.names
response = 'PositiveReview'

gbm_embeddings = H2OGradientBoostingEstimator(stopping_metric = "AUC", stopping_tolerance = 0.001,
                                              stopping_rounds = 5, score_tree_interval = 10,
                                              model_id = "gbm_embeddings.hex"
                                             )
gbm_embeddings.train(x = predictors, y = response, 
                   training_frame = ext_train, validation_frame = ext_test
                  )

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [17]:
print("Baseline AUC: " + str(round(gbm_baseline.auc(valid = True), 3)))
print("With Embeddings AUC: " + str(round(gbm_embeddings.auc(valid = True), 3)))

Baseline AUC: 0.735
With Embeddings AUC: 1.0


In [18]:
# h2o.cluster().shutdown()